# Inscopix python API - imaging processing
### This notebook contains the procedure from preprocessing to ROI detection by CNMFe

### Import packages 
the inscopix python API is located in the same folder of data processing software. The isx.cnmfe is used for ROI detection.

In [1]:
import sys
sys.path.append("/Program Files/Inscopix/Data Processing/")
import os
import isx
import isx.cnmfe

### File path and recording names
Deinfe path and the recording names of our recordings.<br>

In [3]:
path_to_file = 'F:/inscopix_new/'
data_dir = os.path.join(path_to_file, 'BAS0212200417') # Only change here
file_names_in_path = []
file_names_in_path = os.listdir(data_dir)
rec_name = file_names_in_path[4][:-8] # This should be the first/sec raw file in the list with the time stamp
#rec_name = file_names_in_path[4][:-4] # This should be the first/sec raw file in the list with the time stamp


series_rec_names = {
        'day_'+ rec_name[10:18]:
        [
            rec_name,
        ]
}
print(data_dir)
print(series_rec_names)

F:/inscopix_new/BAS0212200417
{'day_20200417': ['recording_20200417_154621']}


### Imgaing Preprocessing, Sptail filter, Motion correction, dF/F, PCA-ICA, Event detection
Note the following cell will perfrom the whole procedure from preprocessing to event detection, it's written in a huge for loop, so keep the identation before all codes.<br>
There will be outputs of file names for each completed steps, so you can check the working progress.

In [4]:
# Process each series all the way to event detection.
output_dir = os.path.join(data_dir, 'processed')
os.makedirs(output_dir, exist_ok=True)

proc_movie_files = []
proc_cs_files = []
for series_name, rec_names in series_rec_names.items():
    
    # Generate the recording file paths.
    rec_files = [os.path.join(data_dir, name + '.xml') for name in rec_names]
    print(rec_files) # for progress feedback, not necessary
    
    # Preprocess the recordings by spatially downsampling by a factor of 4.
    pp_files = isx.make_output_file_paths(rec_files, output_dir, 'PP')
    isx.preprocess(rec_files, pp_files, spatial_downsample_factor=4)
    print(pp_files)
    
    # Perform spatial bandpass filtering with defaults.
    bp_files = isx.make_output_file_paths(pp_files, output_dir, 'BP')
    isx.spatial_filter(pp_files, bp_files, low_cutoff=0.005, high_cutoff=0.500)
    print(bp_files)
    
    # Motion correct the movies using the mean projection as a reference frame.
    mean_proj_file = os.path.join(output_dir, '{}-mean_image.isxd'.format(series_name))
    isx.project_movie(bp_files, mean_proj_file, stat_type='mean')
    mc_files = isx.make_output_file_paths(bp_files, output_dir, 'MC')
    translation_files = isx.make_output_file_paths(mc_files, output_dir, 'translations', 'csv')
    crop_rect_file = os.path.join(output_dir, '{}-crop_rect.csv'.format(series_name))
    isx.motion_correct(
         bp_files, mc_files, max_translation=20, reference_file_name=mean_proj_file,
         low_bandpass_cutoff=None, high_bandpass_cutoff=None,
         output_translation_files=translation_files, output_crop_rect_file=crop_rect_file)
    print(mc_files)
    
    # Run DF/F on the motion corrected movies.
    dff_files = isx.make_output_file_paths(mc_files, output_dir, 'DFF')
    isx.dff(mc_files, dff_files, f0_type='mean')
    print(dff_files)
    
    # Run PCA-ICA on the DF/F movies.
    # Note that you will have to manually determine the number of cells, which we
    # determined here as 180.
    # Increase the block_size to increase speed at the expense of more memory usage.
    ic_files = isx.make_output_file_paths(dff_files, output_dir, 'PCA_ICA')
    num_cells = 180
    isx.pca_ica(dff_files, ic_files, num_cells, int(1.15 * num_cells), block_size=1000)
    #print(ic_files)

    # Run event detection on the PCA-ICA cell sets.
    event_files = isx.make_output_file_paths(ic_files, output_dir, 'ED')
    isx.event_detection(ic_files, event_files, threshold=5)
    print(event_files)

    # Automatically accept and reject cells based on their cell metrics
    # Only accept cells that have a nonzero event rate, an SNR greater
    # than 3, and only one connected component after thresholding
    auto_ar_filters = [('SNR', '>', 3), ('Event Rate', '>', 0), ('# Comps', '=', 1)]
    isx.auto_accept_reject(ic_files, event_files, filters=auto_ar_filters)

    # Store the processed movies and cell sets for longitudinal registration.
    proc_movie_files += dff_files
    proc_cs_files += ic_files
    print(proc_movie_files)

['F:/inscopix_new/BAS0212200417\\recording_20200417_154621.xml']
['F:/inscopix_new/BAS0212200417\\processed\\recording_20200417_154621-PP.isxd']
['F:/inscopix_new/BAS0212200417\\processed\\recording_20200417_154621-PP-BP.isxd']
['F:/inscopix_new/BAS0212200417\\processed\\recording_20200417_154621-PP-BP-MC.isxd']
['F:/inscopix_new/BAS0212200417\\processed\\recording_20200417_154621-PP-BP-MC-DFF.isxd']
['F:/inscopix_new/BAS0212200417\\processed\\recording_20200417_154621-PP-BP-MC-DFF-PCA_ICA-ED.isxd']
['F:/inscopix_new/BAS0212200417\\processed\\recording_20200417_154621-PP-BP-MC-DFF.isxd']


You should see some outputs from each step of precessing above here.


If you have the MC.isxd file already, you can continue with following code.

'F:/inscopix_new/DSC7674200225\\processed\\recording_20200225_153103-PP-BP-MC.isxd'

### ROI detection by CNMFe
Now, we finished all preprocessing from the inscopix, and can move forward for Ca<sup>2</sup> ROI detection by CNMFe. First we would need to define our parameters.

In [4]:
# To start with CNMFe, we can try to set up a cluster on our computer so 
# it can process much faster. Go to anaconda promt, activate isxenv, then run
# ipcluster start -n 16

# Or less amount
# Then use a different terminal to start the jupyter notebook.
# Check again if it's possible to have this at the beginning of the notebook

In [11]:
# Define the motion correct filename for further process.
mov_file = str(mc_files) # This should have the full path to the motion corrected file.

# Create lists of CNMFe output files (cell traces and events)
cellset_list = []
#cellset_list.append(os.path.join(output_dir, mov_file[44:-7]+'-CNMFE_13'+ '.isxd'))
cellset_list.append(os.path.join(output_dir, mov_file[44:-7]+'-CNMFE_17'+ '.isxd'))
#cellset_list.append(os.path.join(output_dir, mov_file[44:-7]+'-CNMFE_21'+ '.isxd'))
for cellset in cellset_list:
    print(cellset)

events_list = []
#events_list.append(os.path.join(output_dir, mov_file[44:-7]+'-CNMFE_13-ED'+ '.isxd'))
events_list.append(os.path.join(output_dir, mov_file[44:-7]+'-CNMFE_17-ED'+ '.isxd'))
#events_list.append(os.path.join(mov_file[2:-7]+'-CNMFE_17-ED'+ '.isxd'))
#events_list.append(os.path.join(output_dir, mov_file[44:-7]+'-CNMFE_21-ED'+ '.isxd'))
for events in events_list:
    print(events)

# remove output files if they already exist
if os.path.exists(cellset_list[0]):
    os.remove(cellset_list[0])
    print('remove '+cellset_list[0])
# if os.path.exists(cellset_list[1]):
#     os.remove(cellset_list[1])
#     print('remove '+cellset_list[1])
# if os.path.exists(cellset_list[2]):
#     os.remove(cellset_list[2])
#     print('remove '+cellset_list[2])

if os.path.exists(events_list[0]):
    os.remove(events_list[0])
    print('remove'+events_list[0])
# if os.path.exists(events_list[1]):
#     os.remove(events_list[1])
#     print('remove'+events_list[1])
# if os.path.exists(events_list[2]):
#     os.remove(events_list[2])
#     print('remove'+events_list[2])

F:/inscopix_new/BES0212200416_test\processed\sed\\recording_20200416_153652-PP-BP-MC-CNMFE_17.isxd
F:/inscopix_new/BES0212200416_test\processed\sed\\recording_20200416_153652-PP-BP-MC-CNMFE_17-ED.isxd


In [34]:
## Test with different setting
# Define the motion correct filename for further process.
mov_file = str(mc_files) # This should have the full path to the motion corrected file.

# Create lists of CNMFe output files (cell traces and events)
cellset_list = []
#cellset_list.append(os.path.join(output_dir, mov_file[44:-7]+'-CNMFE_13'+ '.isxd'))
cellset_list.append(os.path.join(output_dir, mov_file[-41:-7]+'-CNMFE_17'+ '.isxd'))
#cellset_list.append(os.path.join(output_dir, mov_file[44:-7]+'-CNMFE_21'+ '.isxd'))
for cellset in cellset_list:
    print(cellset)

events_list = []
#events_list.append(os.path.join(output_dir, mov_file[44:-7]+'-CNMFE_13-ED'+ '.isxd'))
events_list.append(os.path.join(output_dir, mov_file[-41:-7]+'-CNMFE_17-ED'+ '.isxd'))
#events_list.append(os.path.join(mov_file[2:-7]+'-CNMFE_17-ED'+ '.isxd'))
#events_list.append(os.path.join(output_dir, mov_file[44:-7]+'-CNMFE_21-ED'+ '.isxd'))
for events in events_list:
    print(events)

# remove output files if they already exist
if os.path.exists(cellset_list[0]):
    os.remove(cellset_list[0])
    print('remove '+cellset_list[0])
# if os.path.exists(cellset_list[1]):
#     os.remove(cellset_list[1])
#     print('remove '+cellset_list[1])
# if os.path.exists(cellset_list[2]):
#     os.remove(cellset_list[2])
#     print('remove '+cellset_list[2])

if os.path.exists(events_list[0]):
    os.remove(events_list[0])
    print('remove'+events_list[0])
# if os.path.exists(events_list[1]):
#     os.remove(events_list[1])
#     print('remove'+events_list[1])
# if os.path.exists(events_list[2]):
#     os.remove(events_list[2])
#     print('remove'+events_list[2])

F:/inscopix_new/BES0212200416_test\processed\recording_20200416_153652-PP-BP-MC-CNMFE_17.isxd
F:/inscopix_new/BES0212200416_test\processed\recording_20200416_153652-PP-BP-MC-CNMFE_17-ED.isxd


In [32]:
mov_file[-41:-7]

'recording_20200416_153652-PP-BP-MC'

In [13]:
cellset = 'F:/inscopix_new/BES0212200416_test/processed/recording_20200416_153652-PP-BP-MC-CNMFE_17.isxd'
events = 'F:/inscopix_new/BES0212200416_test/processed/recording_20200416_153652-PP-BP-MC-CNMFE-ED_17.isxd'
print(cellset)
print(events)

F:/inscopix_new/BES0212200416_test/processed/recording_20200416_153652-PP-BP-MC-CNMFE_17.isxd
F:/inscopix_new/BES0212200416_test/processed/recording_20200416_153652-PP-BP-MC-CNMFE-ED_17.isxd


In [19]:
# Test with different name with simple path
cellset = 'F:/CNMFE_17.isxd'
events = 'F:/CNMFE-ED_17.isxd'
print(cellset)
print(events)

F:/CNMFE_17.isxd
F:/CNMFE-ED_17.isxd


In [5]:
## OK I give up now, let's just use it manually.
## Fuck it's still not working...what the heck?

# path_to_file = 'F:/inscopix_new/BES0212200324'
# data_dir = os.path.join(path_to_file, 'processed')
# isx_file_name = 'recording_20200324_103104-PP-BP-MC.isxd'

# mov_file = os.path.join(data_dir, isx_file_name)
# print(mov_file)

# # spatially downsample the movie 2X, making sure to set fix_defective_pixels to False because the movie has already
# # been preprocessed, and we don't want to median filter the movie a second time.
# pp_mov_file = os.path.join(data_dir, 'recording_20200324_103104-PP-BP-MC-PP.isxd')
# if not os.path.exists(pp_mov_file):
#     isx.preprocess(mov_file, pp_mov_file, temporal_downsample_factor=1, 
#                    spatial_downsample_factor=1, fix_defective_pixels=False)
# print(pp_mov_file)


In [6]:
# cellset_list = []
# # cellset_file_gSig3 = os.path.join(data_dir, isx_file_name[:-5]+'-CNMFE_3'+ '.isxd')
# # cellset_file_gSig4 = os.path.join(data_dir, isx_file_name[:-5]+'-CNMFE_4'+ '.isxd')
# # cellset_file_gSig5 = os.path.join(data_dir, isx_file_name[:-5]+'-CNMFE_5'+ '.isxd')
# # print(cellset_file_gSig3)
# cellset_list.append(os.path.join(data_dir, isx_file_name[:-5]+'-CNMFE_13'+ '.isxd'))
# cellset_list.append(os.path.join(data_dir, isx_file_name[:-5]+'-PP-CNMFE_17'+ '.isxd'))
# cellset_list.append(os.path.join(data_dir, isx_file_name[:-5]+'-CNMFE_21'+ '.isxd'))
# print(cellset_list[1])

# events_list = []
# events_list.append(os.path.join(data_dir, isx_file_name[:-5]+'-CNMFE_13-ED'+ '.isxd'))
# events_list.append(os.path.join(data_dir, isx_file_name[:-5]+'-PP-CNMFE_17-ED'+ '.isxd'))
# events_list.append(os.path.join(data_dir, isx_file_name[:-5]+'-CNMFE_21-ED'+ '.isxd'))
# print(events_list[0])

# # events_file_gSig3 = os.path.join(data_dir, isx_file_name[:-5]+'-CNMFE_3-ED'+ '.isxd')
# # events_file_gSig4 = os.path.join(data_dir, isx_file_name[:-5]+'-CNMFE_4-ED'+ '.isxd')
# # events_file_gSig5 = os.path.join(data_dir, isx_file_name[:-5]+'-CNMFE_5-ED'+ '.isxd')

# # remove output files if they already exist
# if os.path.exists(cellset_list[0]):
#     os.remove(cellset_list[0])
#     print('remove '+cellset_list[0])
# if os.path.exists(cellset_list[1]):
#     os.remove(cellset_list[1])
#     print('remove '+cellset_list[1])
# if os.path.exists(cellset_list[2]):
#     os.remove(cellset_list[2])
#     print('remove '+cellset_list[2])

# if os.path.exists(events_list[0]):
#     os.remove(events_list[0])
#     print('remove'+events_list[0])
# if os.path.exists(events_list[1]):
#     os.remove(events_list[1])
#     print('remove'+events_list[1])
# if os.path.exists(events_list[2]):
#     os.remove(events_list[2])
#     print('remove'+events_list[2])

In [70]:
# # new names for outputs (gSig=4)
# cellset_file_3 = cellset_list[1]
# events_file_3 = events_list[1]
# #cellset_file_3
# #print(events_file_3)

In [ ]:
# # Copy from the other script, if it's still not working than I really do know what to do.
# # Change gSig, gSiz, K, min_pnr, rf, stride 
# isx.cnmfe.run_cnmfe(pp_mov_file, cellset_file_3, events_file_3, num_processes=1, K=None, rf=[40, 40], stride=20, 
#                     gSig=4, gSiz=17, min_pnr=10, min_corr=0.8, event_threshold=0.1)

In [21]:
cellset_list[0]
events_list[0]

'F:/inscopix_new/BES0212200416_test\\processed\\sed\\\\recording_20200416_153652-PP-BP-MC-CNMFE_17-ED.isxd'

In [35]:
# CNMFe parameter setting
# Define cell size, with the Ca2+ imaging from the inscopix (w/o crop VoF), 
# a 4x spatial downsampling video usually has a cell size around 15-20 pixel.
# Belows are some parameters that I tried that can give us nice yield of cells (~200-300 cells)

# Most important is the cell size here, change this one.
cell_size_px = 17       # commonly use: 13, 17, 21

#px_set = [13,17,21]     # Use for 
px_set = [17,21]     
gSig=(cell_size_px-1)/4 # gSig
gSiz=cell_size_px       # gSiz = gSig*4+1 (from CNMFe manual)
num_processes=1         # Check again if we can do multi-processor by changing the ipython kernel
K=None
rf=[40,40]
stride=20
min_pnr=10              # Try 20, 30 for the most activative neurons
min_corr=0.8            # Try 0.7 corr
event_threshold=0.3     # Used 0.1 before, too noisy

In [37]:
cellset_list[px_set.index(cell_size_px)]
events_list[px_set.index(cell_size_px)]

'F:/inscopix_new/BES0212200416_test\\processed\\recording_20200416_153652-PP-BP-MC-CNMFE_17-ED.isxd'

In [40]:
os.path.exists(cellset_list[px_set.index(cell_size_px)])

False

In [38]:
## Try again with automatic naming
## Don't use auto naming, skip this one at the moment.

# Run CNMFe
isx.cnmfe.run_cnmfe(mc_files, 
                    cellset_list[px_set.index(cell_size_px)], # Automatic name the output file with corresponding cell size
                    events_list[px_set.index(cell_size_px)],
                    num_processes=num_processes, 
                    K=K, rf=rf, stride=stride,
                    gSig=gSig, gSiz=gSiz, 
                    min_pnr=min_pnr, min_corr=min_corr, 
                    event_threshold=event_threshold,
                    overwrite_tiff=True,
                    output_dir=output_dir)
print('Finished CNMFe '+cellset_list[px_set.index(cell_size_px)])

Exporting .isxd to tiff file(s)...
Wrote .tiff file to: F:/inscopix_new/BES0212200416_test\processed\recording_20200416_153652-PP-BP-MC.tiff


FileNotFoundError: No CaImAn output file found, check the output for errors.

In [20]:
# Run CNMFe
# Now, try to not use the automaic naming.
isx.cnmfe.run_cnmfe(mc_files, 
                    cellset,
                    events,
                    num_processes=num_processes, 
                    K=K, rf=rf, stride=stride,
                    gSig=gSig, gSiz=gSiz, 
                    min_pnr=min_pnr, min_corr=min_corr, 
                    event_threshold=event_threshold)
print('Finished CNMFe')

Exporting .isxd to tiff file(s)...
Wrote .tiff file to: F:/inscopix_new/BES0212200416_test\processed\recording_20200416_153652-PP-BP-MC.tiff


FileNotFoundError: No CaImAn output file found, check the output for errors.

Below is the original code for PCA and ICA, keep it here in case I need them sometime.

In [8]:
# # Process each series all the way to event detection.
# output_dir = os.path.join(data_dir, 'processed')
# os.makedirs(output_dir, exist_ok=True)

# proc_movie_files = []
# proc_cs_files = []
# for series_name, rec_names in series_rec_names.items():
    
#     # Generate the recording file paths.
#     rec_files = [os.path.join(data_dir, name + '.xml') for name in rec_names]
#     print(rec_files) # for progress feedback, not necessary
    
#     # Preprocess the recordings by spatially downsampling by a factor of 4.
#     pp_files = isx.make_output_file_paths(rec_files, output_dir, 'PP')
#     isx.preprocess(rec_files, pp_files, spatial_downsample_factor=4)
#     print(pp_files)
    
#     # Perform spatial bandpass filtering with defaults.
#     bp_files = isx.make_output_file_paths(pp_files, output_dir, 'BP')
#     isx.spatial_filter(pp_files, bp_files, low_cutoff=0.005, high_cutoff=0.500)
#     print(bp_files)
    
#     # Motion correct the movies using the mean projection as a reference frame.
#     mean_proj_file = os.path.join(output_dir, '{}-mean_image.isxd'.format(series_name))
#     isx.project_movie(bp_files, mean_proj_file, stat_type='mean')
#     mc_files = isx.make_output_file_paths(bp_files, output_dir, 'MC')
#     translation_files = isx.make_output_file_paths(mc_files, output_dir, 'translations', 'csv')
#     crop_rect_file = os.path.join(output_dir, '{}-crop_rect.csv'.format(series_name))
#     isx.motion_correct(
#          bp_files, mc_files, max_translation=20, reference_file_name=mean_proj_file,
#          low_bandpass_cutoff=None, high_bandpass_cutoff=None,
#          output_translation_files=translation_files, output_crop_rect_file=crop_rect_file)
#     print(mc_files)
    
#     # Run DF/F on the motion corrected movies.
#     dff_files = isx.make_output_file_paths(mc_files, output_dir, 'DFF')
#     isx.dff(mc_files, dff_files, f0_type='mean')
#     print(dff_files)
    
#     # Run PCA-ICA on the DF/F movies.
#     # Note that you will have to manually determine the number of cells, which we
#     # determined here as 180.
#     # Increase the block_size to increase speed at the expense of more memory usage.
#     ic_files = isx.make_output_file_paths(dff_files, output_dir, 'PCA_ICA')
#     num_cells = 180
#     isx.pca_ica(dff_files, ic_files, num_cells, int(1.15 * num_cells), block_size=1000)
#     #print(ic_files)

#     # Run event detection on the PCA-ICA cell sets.
#     event_files = isx.make_output_file_paths(ic_files, output_dir, 'ED')
#     isx.event_detection(ic_files, event_files, threshold=5)
#     print(event_files)

#     # Automatically accept and reject cells based on their cell metrics
#     # Only accept cells that have a nonzero event rate, an SNR greater
#     # than 3, and only one connected component after thresholding
#     auto_ar_filters = [('SNR', '>', 3), ('Event Rate', '>', 0), ('# Comps', '=', 1)]
#     isx.auto_accept_reject(ic_files, event_files, filters=auto_ar_filters)

#     # Store the processed movies and cell sets for longitudinal registration.
#     proc_movie_files += dff_files
#     proc_cs_files += ic_files
#     print(proc_movie_files)

### Longitudinal registration
Now, we finished all individual file processings, following is the longitudinal registration from different days.
Following cells will also perform event detection and export individual cell maps, csv file of all Ca<sup>2+</sup> traces, registered movies (TIFF) 

In [9]:
# # Perform longitudinal registration on the processed movies and cell sets
# # to align the two days of data.
# lr_cs_files = isx.make_output_file_paths(proc_cs_files, output_dir, 'LR')
# lr_movie_files = isx.make_output_file_paths(proc_movie_files, output_dir, 'LR')
# lr_csv_file = os.path.join(output_dir, 'LR.csv')
# isx.longitudinal_registration(
#     proc_cs_files, lr_cs_files, input_movie_files=proc_movie_files,
#     output_movie_files=lr_movie_files, csv_file=lr_csv_file, accepted_cells_only=True)

In [10]:
# # Then run event detection and automatically classify the cells based on their
# # cell metrics.
# lr_event_files = isx.make_output_file_paths(lr_cs_files, output_dir, 'ED')
# isx.event_detection(lr_cs_files, lr_event_files, threshold=5)
# auto_ar_filters = [('SNR', '>', 3), ('Event Rate', '>', 0), ('# Comps', '=', 1)]
# isx.auto_accept_reject(lr_cs_files, lr_event_files, filters=auto_ar_filters)

In [11]:
# # Finally, export the registered movies, cell sets, and event sets to non-native
# # formats (TIFF and CSV).
# tiff_movie_file = os.path.join(output_dir, 'DFF-LR.tif')
# isx.export_movie_to_tiff(lr_movie_files, tiff_movie_file, write_invalid_frames=True)
# tiff_image_file = os.path.join(output_dir, 'DFF-PCA_ICA-LR.tif')
# csv_trace_file = os.path.join(output_dir, 'DFF-PCA_ICA-LR.csv')
# isx.export_cell_set_to_csv_tiff(lr_cs_files, csv_trace_file, tiff_image_file, time_ref='start')
# csv_event_file = os.path.join(output_dir, 'DFF-PCA_ICA-LR-ED.csv')
# isx.export_event_set_to_csv(lr_event_files, csv_event_file, time_ref='start')